In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import random
import string
import re
#tokenize and remove stopword
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

In [2]:
with open('A:/PWR/project/SMA/News_Category_Dataset_v2.json') as f:
    data = pd.DataFrame(json.loads(line) for line in f)
pd.set_option('max_colwidth', 1000)    
data

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment."""
5,Ron Dicker,ENTERTAINMENT,2018-05-26,Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy,https://www.huffingtonpost.com/entry/morgan-freeman-devastated-sexual-misconduct_us_5b096319e4b0802d69cba298,"""It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor,"" he said in a statement."
6,Ron Dicker,ENTERTAINMENT,2018-05-26,Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit,https://www.huffingtonpost.com/entry/donald-trump-mcondalds-tonight-show_us_5b093561e4b0fdb2aa53daba,"It's catchy, all right."
7,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Amazon Prime That’s New This Week,https://www.huffingtonpost.com/entry/amazon-prime-what-to-watch_us_5b044625e4b0c0b8b23ec14f,There's a great mini-series joining this week.
8,Andy McDonald,ENTERTAINMENT,2018-05-26,Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film,https://www.huffingtonpost.com/entry/mike-myers-reveals-he-wants-to-do-a-fourth-austin-powers-film_us_5b096198e4b0802d69cb9f15,"Myer's kids may be pushing for a new ""Powers"" film more than anyone."
9,Todd Van Luling,ENTERTAINMENT,2018-05-26,What To Watch On Hulu That’s New This Week,https://www.huffingtonpost.com/entry/hulu-what-to-watch_us_5b0445bae4b0c0b8b23ec046,You're getting a recent Academy Award-winning movie.


In [3]:
#data[data['category'] == "CRIME"]

In [4]:
def removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)

In [5]:
author=[]
for a in data['authors']:
    auth=removeNonAscii(a.lower())
    author.append(auth)

In [6]:
categ=[]
for b in data['category']:
    cat=removeNonAscii(b.lower())
    categ.append(cat)

In [7]:
headlines=[]
for c in data['headline']:
    hl=removeNonAscii(c.lower())
    headlines.append(hl)

In [8]:
short=[]
for d in data['short_description']:
    sd=removeNonAscii(d.lower())
    short.append(sd)

In [9]:
clean = pd.DataFrame(dict(authors=author, category=categ, headline=headlines, short_description=short)) 


In [10]:
#clean.to_csv('A:/PWR/project/SMA/dataclean.csv')

In [21]:
dataset = pd.read_csv('A:/PWR/project/SMA/data.csv')
#dataset

In [12]:
#PREPROSESSING
def strip_links(text):
    link_regex    = re.compile("([^-_a-zA-Z0-9!@#%&=,/'\";:~`\$\^\*\(\)\+\[\]\.\{\}\|\?\<\>\\]+|[^\s]+)")
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def drop_digits(in_str):
    digit_list = "1234567890"
    for char in digit_list:
        in_str = in_str.replace(char, "")
    return in_str

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [13]:
headlines2=[]
for c2 in dataset['headline']:
    hl2=strip_all_entities(drop_digits(strip_links(c2)))
    headlines2.append(hl2)

In [14]:
short2=[]
for d2 in dataset['short_desc']:
    sd2=strip_all_entities(drop_digits(strip_links(d2)))
    short2.append(sd2)

In [15]:
preproses = pd.DataFrame(dict(category=dataset.category, headline=headlines2, short_desc=short2)) 
preproses


,category,headline,short_desc
0,business,u s launches auto import probe china vows to defend its interests,the investigation could lead to new u s tariffs similar to those imposed on imported steel and aluminum in march
1,business,starbucks says anyone can now sit in its cafes even without buying anything,the new policy was unveiled weeks after the controversial arrest of two black men at a philadelphia starbucks
2,business,seattle passes controversial new tax on city s biggest companies to combat housing crisis,following the council vote amazons vice president drew herdener said the company has resumed construction planning for its so called block project in downtown seattle
3,business,uber ends forced arbitration in individual cases of sexual assault harassment,victims will be free to go to court but a few caveats remain
4,business,chili s hit by data breach credit and debit card information compromised,the breach is believed to have occurred between march and april
5,business,how uber silences women after sexual assaults,just by using the app you click away your rights now some victims are fighting for change
6,business,how amazon is holding seattle hostage,the city wants to tax large corporations to pay for homeless housing but jeff bezos isn t pleased
7,business,bank of america appears to flip on firearm promise with loan to remington,the bank pledged in april to stop financing companies that sell military style firearms to civilians remington made the bushmaster rifle used in the sandy hook school massacre
8,business,ex volkswagen ceo charged in u s over emissions cheating scandal,martin winterkorn resigned soon after the scandal became public in
9,business,women describe rampant groping sexual harassment at verizon contracted warehouse,verizon launched an investigation soon after huffpost reached out for comment


In [16]:
news_vectorizer = TfidfVectorizer(max_df=0.005, min_df=0.003, stop_words='english')
news_matrix = news_vectorizer.fit_transform(preproses.short_desc)
print 'There are '+ str(news_matrix.get_shape()[0]) + ' documents over a vocabulary feature space of ' + str(news_matrix.get_shape()[1]) + ' terms.'

tfidf = pd.DataFrame(news_matrix.toarray(), index=[preproses.headline, preproses.category], columns= news_vectorizer.get_feature_names())
tfidf

There are 13650 documents over a vocabulary feature space of 307 terms.


,,able,abuse,action,add,administration,age,air,alleged,amazing,anti,...,win,winter,words,workers,working,worth,wrong,wrote,yes,youre
headline,category,,,,,,,,,,,,,,,,,,,,,
u s launches auto import probe china vows to defend its interests,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
starbucks says anyone can now sit in its cafes even without buying anything,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
seattle passes controversial new tax on city s biggest companies to combat housing crisis,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uber ends forced arbitration in individual cases of sexual assault harassment,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chili s hit by data breach credit and debit card information compromised,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
how uber silences women after sexual assaults,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
how amazon is holding seattle hostage,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bank of america appears to flip on firearm promise with loan to remington,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ex volkswagen ceo charged in u s over emissions cheating scandal,business,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
training_data=tfidf.iloc[:-350]
training_data

,,able,abuse,action,add,administration,age,air,alleged,amazing,anti,...,win,winter,words,workers,working,worth,wrong,wrote,yes,youre
headline,category,,,,,,,,,,,,,,,,,,,,,
u s launches auto import probe china vows to defend its interests,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
starbucks says anyone can now sit in its cafes even without buying anything,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
seattle passes controversial new tax on city s biggest companies to combat housing crisis,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
uber ends forced arbitration in individual cases of sexual assault harassment,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
chili s hit by data breach credit and debit card information compromised,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
how uber silences women after sexual assaults,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
how amazon is holding seattle hostage,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
bank of america appears to flip on firearm promise with loan to remington,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
ex volkswagen ceo charged in u s over emissions cheating scandal,business,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [18]:
training_label=preproses.category

In [27]:
testing_data=tfidf.iloc[-350:]
testing_data

,,able,abuse,action,add,administration,age,air,alleged,amazing,anti,...,win,winter,words,workers,working,worth,wrong,wrote,yes,youre
headline,category,,,,,,,,,,,,,,,,,,,,,
european debt crisis in spain soaring joblessness fuels barter economy,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jamie dimon beware the wall street journal says it is time to break up the big banks,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
influencer of the week jeremy lin,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the disingenuous economics of ron paul,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
why inequality matters the housing crisis our justice system capitalism,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cancer screening budget cuts may prove deadly for some women,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
high deductible health insurance pinches workers from both sides,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
american apparel cracks down on shoplifting loses hipster fans,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
leadership eavesdropping and muffins,business,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
